In [3]:
import numpy as np
import pandas as pd
import gc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task


'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


/home/zhenghao/.conda/envs/light_new/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zhenghao/.conda/envs/light_new/lib/python3.9/site-packages/lightautoml/ml_algo/dl_model.py:42: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/home/zhenghao/.conda/envs/light_new/lib/python3.9/site-packages/lightautoml/text/embed.py:22: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/home/zhenghao/.conda/envs/light_new/lib/python3.9/site-packages/lightautoml/text/dl_transformers.py:25: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")


In [4]:
train = pd.read_csv('/home/zhenghao/kaggle/train.csv')

In [5]:
X_train, X_val = train_test_split(train, test_size=0.2, random_state=42, shuffle=True, stratify=train.Response)

In [7]:
task = Task('binary') #‘binary’ 
automl = TabularAutoML(
    task = task, 
    timeout = 600 * 3600,
    cpu_limit = 12,
    gpu_ids = '0',
    general_params = {"use_algos": [['denselight']]},
    nn_params = {
        "n_epochs": 10, 
        "bs": 1024, 
        "num_workers": 2, 
        "path_to_save": None, 
        "freeze_defaults": True,
        "cont_embedder": 'plr',
        'cat_embedder': 'weighted',
        'act_fun': 'SiLU',
        "hidden_size": [512, 128], #32,
        'stop_by_metric': True,
        'embedding_size': 32,
        'verbose_bar': True,
        "snap_params": { 'k': 2, 'early_stopping': True, 'patience': 1, 'swa': True }, 
        'opt_params': { 'lr': 0.0003 , 'weight_decay': 0 }
    },
    nn_pipeline_params = {"use_qnt": True, "use_te": False},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42, 'advanced_roles': True}
)

out_of_fold_predictions = automl.fit_predict(
    X_train, valid_data=X_val,
    roles = {
        'target': 'Response',
        'drop': ['id'],
    }, 
    verbose = 4
)

[14:00:45] Stdout logging level is DEBUG.
[14:00:45] Task: binary

[14:00:45] Start automl preset with listed constraints:
[14:00:45] - time: 2160000.00 seconds
[14:00:45] - CPU: 12 cores
[14:00:45] - memory: 16 GB

[14:00:45] Train data shape: (9203838, 12)

[14:00:59] Feats was rejected during automatic roles guess: []
[14:01:00] Layer 1 train process start. Time left 2159985.63 secs
[14:01:23] number of text features: 0 
[14:01:23] number of categorical features: 8 
[14:01:23] number of continuous features: 2 


val: 100%|██████████| 2248/2248 [00:23<00:00, 95.93it/s] 


[14:04:38] Epoch: 0, train loss: 0.24925921857357025, val loss: 0.24456407129764557, val metric: 0.8908973169590448


val: 100%|██████████| 2248/2248 [00:23<00:00, 94.75it/s] 


[14:07:51] Epoch: 1, train loss: 0.24294474720954895, val loss: 0.24324508011341095, val metric: 0.8918149902828341


...

[14:14:21] Epoch: 3, train loss: 0.23956945538520813, val loss: 0.2429785132408142, val metric: 0.8920336993743323


val: 100%|██████████| 2248/2248 [00:23<00:00, 94.69it/s] 


[14:17:38] Epoch: 4, train loss: 0.2382112294435501, val loss: 0.24312721192836761, val metric: 0.8918410049115594


val: 100%|██████████| 2248/2248 [00:23<00:00, 94.76it/s]


[14:18:03] Early stopping: val loss: 0.24261802434921265, val metric: 0.8923466688626919
[14:18:03] Lvl_0_Pipe_0_Mod_0_TorchNN_denselight_0 fitting and predicting completed
[14:18:03] Time left 2158962.23 secs

[14:18:03] Layer 1 training completed.

[14:18:03] Automl preset training completed in 1037.81 seconds

[14:18:03] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (1 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN_denselight_0) 



In [8]:
del train, X_train, X_val
gc.collect()

0

In [9]:
test = pd.read_csv('/home/zhenghao/kaggle/test.csv')
ss = pd.read_csv('/home/zhenghao/kaggle/sample_submission.csv')

In [10]:
pred = automl.predict(test).data[:,  0]

ss.Response = pred
ss.to_csv('LightAutoML_denselight_1fold.csv', index=None)
ss

test: 100%|██████████| 7491/7491 [00:47<00:00, 157.30it/s]


,id,Response
0,11504798,0.003658
1,11504799,0.672376
2,11504800,0.228664
3,11504801,0.000038
4,11504802,0.144176
...,...,...
7669861,19174659,0.240385
7669862,19174660,0.000112
7669863,19174661,0.000143
7669864,19174662,0.669437
